# Run a gas network simulation

This notebook shows how the **IntegrCiTy Data Access Layer** (DAL) can be used to execute a simulation setup stored in the 3DCityDB and how to store the results back to the 3DCityDB. 

## Define connection parameters for database

First, we define the connection parameters for the 3DCityDB:

In [1]:
from dblayer import *

connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
)

## Retrieve co-simulation graph from the database

For retrieving ZerOBNL simulation setups from the database, [package dblayer](https://github.com/IntegrCiTy/dblayer) provides a dedicated helper class (*dblayer.zerobnl*):

In [2]:
from dblayer.zerobnl.reader import *

reader = DBReader( connect )
sim = reader.read_from_db( 'GasGridSim' )

To check that the co-simulation setup has been loaded correctly, we can for instance take a look at the retrived nodes:

In [3]:
sim.nodes

,ToSet,ToGet,InitVal,Parameters,Wrapper,Dockerfile,Files,Local,Meta,Env
GasGrid,"[(gas_node47/p_kW, kW), (gas_node48/p_kW, kW),...",[],"{'network_id': 3000, 'dbport': 5432, 'dbname':...",{},/c/Development/integrcity/DemoToolchain/3_simu...,/c/Development/integrcity/DemoToolchain/3_simu...,[],False,GasGridMeta,GasGridEnv
Consumers,[],"[(gas_node47, kW), (gas_node48, kW), (gas_node...",{'start_date': '2020-01-01 00:00:00+00:00'},{},/c/Development/integrcity/DemoToolchain/3_simu...,/c/Development/integrcity/DemoToolchain/3_simu...,[/c/Development/integrcity/DemoToolchain/3_sim...,False,ConsumersMeta,ConsumersEnv


## Make the 3DCityDB available in the simulation

[ZerOBNL](https://github.com/IntegrCiTy/zerobnl) runs all parts of the co-simulation in separate Docker containers.
Because both nodes will try to connect to the 3DCityDB during initialization, we need to add the Docker container running the 3DCityDB to the simulation setup:

In [4]:
sim.add_container_to_simulation( 'citydb-container' )

*Note: This notebook assumes that you run the 3DCityDB in a Docker container (see the [README](../README.md) file).
In case you run the 3DCityDB in another way, you will have to make sure that it is available via the* host *and* port *provided in the simulation setup from* inside *the simulation containers. There are several ways to do that, one is to add a proxy container that routes the connection accordingly.*

## Running the simulation

Simple execute the *run* method.

**NOTE**: This may take a few miniutes, because the calculation of the network involves a lot of number crunching ...

In [5]:
sim.run()

INFO :: Simulation finished in 31 min and 1 sec


## Retrieve the simulation results

After the simulation has finished, connect to ZerOBNL and retrieve the results:

In [6]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,X,GasGrid,gas_pipe17-262/m_dot
1,X,GasGrid,gas_node157/P
2,X,GasGrid,gas_node99/P
3,OUT,Consumers,gas_node84
4,X,GasGrid,gas_node227/P
...,...,...,...
682,X,GasGrid,gas_pipe117-174/m_dot
683,X,GasGrid,gas_node260/P
684,X,GasGrid,gas_pipe217-219/m_dot
685,X,GasGrid,gas_node255/P


Retrieve all the results that were produced by the gas network simulator (node *GasGrid*):

In [7]:
import logging
logging.getLogger("ZEROBNL").setLevel(logging.WARNING)

results = sim.get_results_by_pattern("X||GasGrid||*")
print( 'Found {} results.'.format( len( results ) ) )

Found 564 results.


# Write results to database

Next, we write the simulation results back to the 3DCityDB for futher analysis (see next notebook).

## Extract results for individual pipes

To do so, we simple iterate through the results and extract the ones we want to analyze further to a dict.
In this case, we want to look at the mass flow rate in the pipes, which all contain the string *m_dot* in their name:

In [8]:
pipes_m_dot = {}

for name, ts in results.items():
    if 'm_dot' in name:
        sim_var_name = name.split('||')[2]
        pipe_name = sim_var_name.split('/')[0]
        pipes_m_dot[pipe_name] = list(ts.array)

## Retrieve information about pipes stored in database

The goal is to associate the simulation results to the network data already stored in the 3DCityDB.
Hence, we connect to the 3DCityDB:

In [9]:
db_access = DBAccess()
db_access.connect_to_citydb(connect)

Now we can retrieve all the pipes belonging to the gas network.
For this, we can make a query, that asks for all pipes belonging to the network with the correct ID.
To do so, we start again by retrieving all the definitions of the relevant data types:

In [10]:
# Get definition of type "RoundPipe" from view "utn9_ntw_feat_distrib_elem_pipe_round". 
RoundPipe = db_access.map_citydb_object_class( 
    'RoundPipe', 
    table_name='utn9_ntw_feat_distrib_elem_pipe_round', schema='citydb_view' 
)

# Get definition of linking between objects of type "NetworkFeature" to objects 
# of type "Network" from table "utn9_network_to_network_feature". 
# This linking has no official name, so we call it "NetworkToFeature".
NetworkToFeature = db_access.map_citydb_object_class( 
    'NetworkToFeature', 
    table_name='utn9_network_to_network_feature', schema='citydb', user_defined = True
)

Now we can make the actual query to retrieve the data:

In [11]:
pipes_db_data = db_access.join_citydb_objects(
    [ 'RoundPipe', 'NetworkToFeature' ], 
    conditions = [
        # These conditions make sure that we only retrieve pipes belonging to the network with ID=3000.
        RoundPipe.id == NetworkToFeature.network_feature_id,
        NetworkToFeature.network_id == 3000        
        ]
    )

## Associate results to pipes stored in database

We will use the [Scenario ADE](https://github.com/gioagu/3dcitydb_scenario_ade) to associate the simulation results (mass flow rates) with objects (pipes).
The Scenario ADE is a domain extension for the 3DCityDB for storing, representing, and managing entities and attributes related to specific scenarios (such as the results of a specific simulation).

Package dblayer provides helper functions and classes (*dblayer.helpers.scn*) for storing time series to the 3DCityDB and associating it with other objects in the database.
In this specific example, we collect the mass flow rate results and the link to the corresponding pipes with the help of instances of class *SimResultRegularTimeSeries*:

In [12]:
import datetime
from dblayer.helpers.scn.add_sim_results import *

# We will fill this list with the data that will be added to the 3DCityDB.
sim_res_ts = []

# Iterate through all pipes.
for data in pipes_db_data:
    
    # Retrieve pipe data and corresponding simulation results.
    pipe = data[0]
    val_array_m_dot = pipes_m_dot[pipe.name]
    
    # Put together all relevant information in an instance of class "SimResultRegularTimeSeries".
    ts_m_dot = SimResultRegularTimeSeries(
        name = '{}_m_dot'.format(pipe.name),
        # Provide link to pipe via its object ID.
        object_id = pipe.id,
        # Provide the actual time series data and all relevant information about it.
        values_array = val_array_m_dot,
        values_unit = 'kg/s',
        temporal_extent_begin = datetime.datetime( 2020, 1, 1, 0, 0, 0 ),
        temporal_extent_end = datetime.datetime( 2020, 1, 1, 11, 0, 0 ),
        time_interval = 1,
        time_interval_unit = 'h',
        acquisition_method = 'Simulation',
        interpolation_type = 'AverageInSucceedingInterval',
        )

    # Add this to the list of data to be added to the 3DCityDB.
    sim_res_ts.append(ts_m_dot)

Now we can use function *add_sim_results_regular_time_series* to add these results to the 3DCityDB:

In [13]:
add_sim_results_regular_time_series(
    db_access,
    'GasGridSimResults',
    sim_res_ts
    )

1

## Save the results to the database

Above, the data was *added* to the database session. In order to make it persistent, i.e., to store it permanently in the database, it has to be *committed* to the 3DCityDB.
This is done via *commit_citydb_session*:

In [14]:
db_access.commit_citydb_session()

Finally, delete the instance of class DBAccess to close the session.

In [15]:
del db_access

Next up is notebook [4_visualize_sim_results.ipynb](../4_visualize/4_visualize_sim_results.ipynb), which demonstrates how to extract data from the 3DCItyDB and visualize it.